In [1]:
K.<a> = NumberField(x - 1)
from sage.rings.polynomial.cyclotomic import cyclotomic_coeffs
S = ZZ['x']
K.<w> = NumberField(x^2 - x + 1)
R.<x> = PolynomialRing(ZZ)



def is_relatively_prime(m,n):
    if gcd(m,n) ==1:
        return 1
    else:
        return 0 
    
def raw_fekete(n):
    "Fekete polynomial with Delta odd "
    D=n
    v=[is_relatively_prime(a+1,n) for a in range(0,D-1)]
    F_D=R(v)
    return F_D*x  

def fekete_by_last_factor(n):
    F=raw_fekete(n)
    f = F.factor()[-1][0]
    return f

def divisor_set_general(q,p):
    divisor = set()
    for d in range(2, q):
        if q%d ==1:
            divisor.add(d)
    for d in range(2,p):
        if p%d==1 and d !=q:
            divisor.add(d)
    for d in range(2,p+q+1):
        if (q*p+1)%d==0 and (p+q)%d==0:
            divisor.add(d)
    return divisor 
   
    
def cyclotomic_factor(q,p):
    """
    return the cyclotmic factor of F_{qp}
    """
    cyc_factor =x
    divisor_set = divisor_set_general(q,p)
    for d in divisor_set:
        cyc_factor *=S(cyclotomic_coeffs(d))
    return cyc_factor


def fekete(q,p):
    n=p*q
    cyc_factor = cyclotomic_factor(q,p)
    F= raw_fekete(n)
    f,r =F.quo_rem(cyc_factor) 
    return f 


def reduced_fekete(f):
    u=f.trace_polynomial()
    g_D=u[0]
    return g_D

def fekete_reduction(f, q):
    f=f.change_ring(GF(q))
    return f.factor()        


def two_four_cycle(f,q):
    factor = fekete_reduction(f,q)
    number_of_factor = len(factor)
    #check that f is separable modulo q
    for i in range(number_of_factor):
        if factor[i][1] >1:
            return False
    number_two_cycle =0 
    number_four_cycle =0 
    number_even_cycle =0
    for i in range(number_of_factor):
        if factor[i][0].degree() ==2:
            number_two_cycle +=1
        if factor[i][0].degree() == 4:
            number_four_cycle +=1 
        if factor[i][0].degree() % 2 ==0:
            number_even_cycle +=1
    if number_two_cycle ==1 and number_four_cycle ==1 and number_even_cycle ==2:
        return True
    return False
            
    
def search_two_four_cycle(f,n):
    for q in range(n):
        if is_prime(q):
            if two_four_cycle(f,q):
                return q
    return -1 

def search_two_four_cycle_2(f,m,n):
    for q in range(m,n):
        if is_prime(q):
            if two_four_cycle(f,q):
                return q
    return -1 



def almost_cycle(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==3: 
                factor1=factor[0][0]
                factor2=factor[1][0]
                degree1=factor1.degree()
                degree2=factor2.degree()
                if degree1==1 and degree2==1 and factor[0][1]==1 and factor[1][1]==1 and factor[2][1]==1: 
                    return q
    return  -1         
                
def irreducible(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==1 and factor[0][1]==1:
                    return q
    return  -1         
                
       
    
def length_test_2(v):
    #count the number of even entries in v
    count2=0
    for item in v:
        if item==2:
            count2 +=1
    count_even=0     
    for item in v:        
        if item %2 ==0:
            count_even +=1
    if count2==count_even==1:
        return True
    return False    

def length_test_4(v):
    #count the number of even entries in v
    count4=0
    for item in v:
        if item==4:
            count4 +=1
    count_even=0     
    for item in v:        
        if item %2 ==0:
            count_even +=1
    if count4==count_even==1:
        return True
    return False    

    
def two_cycle(f,n):
    result=[]
    for q in range(n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_2(v):
            return q
    return -1    

def two_cycle_2(f,m,n):
    result=[]
    for q in range(m,n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_2(v):
            return q
    return -1 

def four_cycle(f,n):
    result=[]
    for q in range(n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_4(v):
            return q
    return -1  

def cycle(g,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(g,q)
            if len(factor)==2: 
                factor1=factor[0][0]
                coef=factor1.degree()
                if coef==1 and factor[0][1]==1 and factor[1][1]==1: 
                                   return q
    return  -1   
                    

                    
def search_quadruple(f,n):
    irr=irreducible(f,n)
    print(f"f is irreducible at q= ", irr)
    q_cycle=almost_cycle(f,n)
    print(f"f has an (2m-2) cycle at q=", q_cycle)
    q_tranposition=two_cycle(f,n)
    print(f"f has an 2-cycle at q=", q_tranposition)
    q_four_cycle=four_cycle(f,n)
    print(f"f has an 4-cycle at q=", q_four_cycle)
    

def quadruple(f,n):
    irr=irreducible(f,n)
    q_cycle=almost_cycle(f,n)
    q_tranposition=two_cycle(f,n)
    q_four_cycle=four_cycle(f,n)
    result=(irr, q_cycle, q_tranposition, q_four_cycle)
    return result
    
    
def triple(g,n):
    irr=irreducible(g,n)
    q_cycle=cycle(g,n)
    q_tranposition=two_cycle(g,n)
    result=[irr, q_cycle, q_tranposition]
    return result

In [4]:
q=11
p=13
f=fekete(11, 13)
g=reduced_fekete(f)
g

x^55 - 56*x^53 + 1483*x^51 - 24700*x^49 + 290276*x^47 - 2559808*x^45 + x^44 + 17586788*x^43 - 44*x^42 - 96474928*x^41 + 901*x^40 + 429651014*x^39 - 11400*x^38 - 1571131888*x^37 + 99789*x^36 + 4752957026*x^35 - 641172*x^34 - 11949259336*x^33 + 3131129*x^32 + 25017769297*x^31 - 11872290*x^30 - 43615970335*x^29 + 35403145*x^28 + 63181993729*x^27 - 83590294*x^26 - 75725722320*x^25 + 156577615*x^24 + 74604393370*x^23 - 232168714*x^22 - 59868635449*x^21 + 270828358*x^20 + 38657357873*x^19 - 246018191*x^18 - 19761625453*x^17 + 171539468*x^16 + 7827795044*x^15 - 90114246*x^14 - 2334313385*x^13 + 34883252*x^12 + 503701720*x^11 - 9714654*x^10 - 74323349*x^9 + 1897413*x^8 + 6882014*x^7 - 248485*x^6 - 345597*x^5 + 19042*x^4 + 7005*x^3 - 633*x^2 - 36*x + 5

In [6]:
fekete_reduction(g, 17)

x^55 + 12*x^53 + 4*x^51 + x^49 + x^47 + x^45 + x^44 + 16*x^43 + 7*x^42 + 4*x^41 + x^39 + 7*x^38 + 3*x^37 + 16*x^36 + 7*x^35 + x^32 + x^31 + 3*x^29 + 16*x^28 + 7*x^27 + 15*x^26 + 15*x^25 + 16*x^24 + 11*x^23 + 14*x^22 + 14*x^21 + 15*x^20 + 14*x^19 + 12*x^18 + 16*x^17 + 16*x^16 + 16*x^13 + 16*x^11 + 13*x^10 + 5*x^9 + 9*x^8 + 6*x^7 + 4*x^6 + 13*x^5 + 2*x^4 + x^3 + 13*x^2 + 15*x + 5

In [7]:
fekete_reduction(g, 353)

(x + 106) * (x^54 + 247*x^53 + 237*x^52 + 294*x^51 + 324*x^50 + 250*x^49 + 338*x^48 + 178*x^47 + 304*x^46 + 252*x^45 + 264*x^44 + 257*x^43 + 267*x^42 + 247*x^41 + 265*x^40 + 345*x^39 + 30*x^38 + 246*x^37 + 205*x^36 + 46*x^35 + 241*x^34 + 99*x^33 + 89*x^32 + 116*x^31 + 305*x^30 + 305*x^29 + 281*x^28 + 288*x^27 + 158*x^26 + 302*x^25 + 171*x^24 + 106*x^23 + 260*x^22 + 66*x^21 + 117*x^20 + 4*x^19 + 218*x^18 + 54*x^17 + 350*x^16 + 142*x^15 + 50*x^14 + 295*x^13 + 220*x^12 + 123*x^11 + 42*x^10 + 43*x^9 + 126*x^8 + 96*x^7 + 340*x^6 + 346*x^5 + 26*x^4 + 48*x^3 + 152*x^2 + 199*x + 50)

In [8]:
fekete_reduction(g, 19)

(x^2 + 10*x + 11) * (x^9 + 8*x^8 + 4*x^7 + 6*x^6 + 18*x^5 + 3*x^4 + 8*x^3 + 8*x^2 + x + 8) * (x^17 + 14*x^16 + 5*x^15 + 2*x^14 + 13*x^13 + 12*x^12 + 5*x^11 + 17*x^10 + 7*x^9 + 12*x^8 + 12*x^7 + 3*x^6 + 8*x^5 + 8*x^4 + 11*x^3 + 15*x^2 + 7*x + 17) * (x^27 + 6*x^26 + 8*x^25 + 9*x^24 + 11*x^23 + 18*x^22 + 18*x^21 + 13*x^20 + 13*x^19 + 18*x^18 + 18*x^17 + 6*x^16 + 16*x^15 + 18*x^14 + 5*x^13 + 13*x^12 + 10*x^11 + 4*x^10 + 16*x^9 + 8*x^8 + 12*x^7 + 18*x^6 + 14*x^5 + 10*x^4 + 2*x^3 + 7*x^2 + 4*x + 18)

# Galois groups of $g_{11p}$

In [2]:
%%time
P=Primes()
n=10**5
q=11
p=q
while p < 100: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

13 [17, 353, 19]


17 [1399, 347, 103]


19 [257, 173, 47]


23 [1319, 431, 397]


29 [1009, 643, 19]


...

79 [1229, 3023, 739]


83 [167, 10301, 29]


89 [71, 1831, 677]


97 [1487, 2207, 373]


101 [443, 7877, 311]
CPU times: user 5min 17s, sys: 1.25 s, total: 5min 18s
Wall time: 10min 52s


In [9]:
%%time
P=Primes()
n=10**5
q=11
p=101
while p < 200: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

103 [5413, 6217, 241]


107 [421, 157, 179]


109 [5009, 5591, 487]


113 [2039, 1439, 389]


127 [8839, 6067, 797]


...

191 [821, 1373, 73]


193 [20959, 18859, 19]


197 [32057, 9161, 433]


199 [2741, 6917, 1063]


211 [73, 1877, 67]
CPU times: user 1h 40min 4s, sys: 15.9 s, total: 1h 40min 20s
Wall time: 3h 27min 29s


In [10]:
%%time
P=Primes()
n=10**5
q=11
p=211
while p < 300: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

223 [661, 2027, 103]


227 [26237, 1783, 661]


229 [6899, 10243, 29]


233 [853, 36097, 23]


239 [7109, 2381, 73]


...

277 [18289, 307, 1283]


281 [631, 2633, 811]


283 [23, 3083, 109]


293 [22639, 2437, 139]


307 [1429, 6079, 139]
CPU times: user 5h 41min 32s, sys: 37.5 s, total: 5h 42min 10s
Wall time: 11h 52min 23s


In [11]:
%%time
P=Primes()
n=10**5
q=11
p=307
while p < 400: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

311 [2251, 2731, 937]


313 [4583, 11593, 191]


317 [269, 13697, 383]


331 [139, 17497, 241]


337 [1709, 61643, 2029]


...

379 [1259, 331, 1049]


383 [5569, 4567, 199]


389 [14563, 15817, 23]


397 [1583, 4801, 157]


401 [24709, 6311, 547]
CPU times: user 9h 40min 36s, sys: 54.5 s, total: 9h 41min 31s
Wall time: 15h 23min 11s


In [12]:
%%time
P=Primes()
n=10**5
q=11
p=401
while p < 450: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

409 [4447, 6871, 1367]


419 [55997, 39313, 787]


421 [8609, 7129, 317]


431 [1619, 19861, 751]


433 [11173, 3299, 17]


439 [29983, 20627, 149]


443 [5303, 26863, 2297]


449 [19079, 733, 47]


457 [1741, 10771, 37]
CPU times: user 11h 46min 48s, sys: 50.4 s, total: 11h 47min 38s
Wall time: 12h 1min 54s


In [13]:
%%time
P=Primes()
n=10**5
q=11
p=457
while p < 500: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

461 [1399, 31153, 479]


463 [10141, 11321, 83]


467 [47713, 15359, 617]


479 [40801, 3911, 1361]


487 [36263, 28859, 151]


491 [21613, 41411, 919]


499 [63211, 1907, 101]


503 [8101, 80021, 863]
CPU times: user 1d 2h 36min 52s, sys: 2min 25s, total: 1d 2h 39min 17s
Wall time: 1d 21h 34min 59s


 # Galois group of $f_{11p}$.

First, we consider the case $p \equiv 1 \pmod{11}$. In this case, the discriminant of $f_{11p}$ is not a perfect square. To show that the Galois group of $f_{11p}$ is $(\Z/2)^n \rtimes S_n$ where $\deg(f_{11p})=2n$, we need to show that the Galois group of $f_{11p}$ contains a 2-cycle. The following codes find the smallest prime $q$ such that the reduction of $f_{11p}$ modulo $q$ produces a 2-cycle. 

In [14]:
%%time
P=Primes()
n=10**5
q=11
p=11
while p < 100: 
    p=P.next(p)
    if p % q ==1: 
        f = fekete(q,p)
        print(p, two_cycle(f,n))

23 431


67 3329


89 4583
CPU times: user 8min 55s, sys: 1.12 s, total: 8min 56s
Wall time: 18min 12s


In [15]:
%%time
P=Primes()
n=10**5
q=11
p=89
while p < 200: 
    p=P.next(p)
    if p % q ==1: 
        f = fekete(q,p)
        print(p, two_cycle(f,n))

199 7937
CPU times: user 1h 14min 48s, sys: 9.13 s, total: 1h 14min 57s
Wall time: 2h 32min 42s


In [17]:
%%time
P=Primes()
n=10**5
q=11
p=200
while p < 400: 
    p=P.next(p)
    if p % q ==1: 
        f = fekete(q,p)
        print(p, two_cycle(f,n))

331 193


353 4231


397 34033
CPU times: user 1d 10h 25min 37s, sys: 4min 3s, total: 1d 10h 29min 41s
Wall time: 2d 20h 57min 3s


In [18]:
%%time
P=Primes()
n=10**5
q=11
p=400
while p < 500: 
    p=P.next(p)
    if p % q ==1: 
        f = fekete(q,p)
        print(p, two_cycle(f,n))

419 39313


463 4397
CPU times: user 2d 5h 2min 7s, sys: 7min 39s, total: 2d 5h 9min 46s
Wall time: 5d 13h 26min 14s


We next consider the case $p \not \equiv 1 \pmod{11}$. In this case, the discriminant of $f_{11p}$ is a perfect square. To verify that it is maximal, we need to find a 2-4 cycle. 

In [19]:
%%time
P=Primes()
n=10**5
q=11
p=11
while p < 100: 
    p=P.next(p)
    if p % q !=1: 
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

13 1423


17 2081


19 47


29 1873


31 1861


...

73 7457


79 10993


83 17579


97 20173


101 1811
CPU times: user 2h 41min 34s, sys: 30.8 s, total: 2h 42min 5s
Wall time: 8h 19min 52s


In [20]:
%%time
P=Primes()
n=10**5
q=11
p=101
while p < 200: 
    p=P.next(p)
    if p % q !=1: 
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

103 3671


107 2699


109 53549


113 35677


127 23059


...

181 5683


191 17851


193 36013


197 27427


211 2719
CPU times: user 2d 1h 33min 35s, sys: 8min 22s, total: 2d 1h 41min 58s
Wall time: 6d 8h 12s


In [21]:
%%time
P=Primes()
n=10**5
q=11
p=211
while p < 300: 
    p=P.next(p)
    if p % q !=1: 
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

223 82013


227 5527


229 3407


233 5323


239 3407


...

277 23057


281 7517


283 4549


293 3967


307 24527
CPU times: user 5d 22h 32min 47s, sys: 22min 23s, total: 5d 22h 55min 11s
Wall time: 13d 17h 3min 26s
